In [ ]:
pip install -r requirements.txt -q

In [ ]:
import requests
import pandas as pd
import json
import xml.etree.ElementTree as ET

ARXIV_NAMESPACE = '{http://www.w3.org/2005/Atom}'

def extract_from_arxiv(search_query='cat:cs.AI', max_results=100, json_file_path='files/arxiv_dataset.json'):
    """
    Fetches papers from the ArXiv API based on a search query, saves them as JSON,
    and returns a pandas DataFrame.

    Args:
        search_query (str): The search query for ArXiv (default is 'cat:cs.AI').
        max_results (int): The maximum number of results to retrieve (default is 100).
        json_file_path (str): File path where JSON data will be saved.

    Returns:
        pd.DataFrame: DataFrame containing the extracted paper information.
    """

    url = f'http://export.arxiv.org/api/query?search_query={search_query}&max_results={max_results}'
    response = requests.get(url)
    root = ET.fromstring(response.content)

    papers = []

    # Loop through each "entry" in the XML, representing a single paper.
    for entry in root.findall(f'{ARXIV_NAMESPACE}entry'):
        title = entry.find(f'{ARXIV_NAMESPACE}title').text.strip()
        summary = entry.find(f'{ARXIV_NAMESPACE}summary').text.strip()

        author_elements = entry.findall(f'{ARXIV_NAMESPACE}author')
        authors = [author.find(f'{ARXIV_NAMESPACE}name').text for author in author_elements]
        paper_url = entry.find(f'{ARXIV_NAMESPACE}id').text
        arxiv_id = paper_url.split('/')[-1]
        pdf_link = next((link.attrib['href'] for link in entry.findall(f'{ARXIV_NAMESPACE}link')
                         if link.attrib.get('title') == 'pdf'), None)

        papers.append({
            'title': title,
            'summary': summary,
            'authors': authors,
            'arxiv_id': arxiv_id,
            'url': paper_url,
            'pdf_link': pdf_link
        })

    # Convert list into a pandas DataFrame.
    df = pd.DataFrame(papers)

    # Save the DataFrame to a JSON file.
    with open(json_file_path, 'w', encoding='utf-8') as f:
        json.dump(papers, f, ensure_ascii=False, indent=4)
        print(f'Data saved to {json_file_path} ...')

    return df


In [ ]:
df = extract_from_arxiv(max_results=20)

In [ ]:
import json
file_name = 'files/arxiv_dataset.json'
with  open(file_name, 'r') as file:
    data = json.load(file)

print(data)

In [ ]:
import pandas as pd
df = pd.DataFrame(data)
df.sample(n=5)

In [ ]:
import pandas as pd
import requests
import os

def download_pdfs(df, download_folder='files'):
    """
    Downloads PDFs from URLs listed in the DataFrame and saves them to a specified folder.
    The file names are stored in a new column 'pdf_file_name' in the DataFrame.

    Args:
        df (pd.DataFrame): DataFrame containing a 'pdf_link' column with URLs to download.
        download_folder (str): Path to the folder where PDFs will be saved (default is 'files').

    Returns:
        pd.DataFrame: The original DataFrame with an additional 'pdf_file_name' column containing
                      the paths of the downloaded PDF files or None if the download failed.
    """

    if not os.path.exists(download_folder):
        os.makedirs(download_folder)

    pdf_file_names = []

    # Loop through each row to download PDFs
    for index, row in df.iterrows():
        pdf_link = row['pdf_link']

        try:
            response = requests.get(pdf_link)
            response.raise_for_status()

            file_name = os.path.join(download_folder, pdf_link.split('/')[-1]) + '.pdf'
            pdf_file_names.append(file_name)

            with open(file_name, 'wb') as f:
                f.write(response.content)

            print(f'PDF downloaded successfully and saved as {file_name}')

        except requests.exceptions.RequestException as e:
            print(f'Failed to download the PDF: {e}')
            pdf_file_names.append(None)

    df['pdf_file_name'] = pdf_file_names

    return df


In [ ]:
df = download_pdfs(df)

In [ ]:
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

def load_and_chunk_pdf(pdf_file_name, chunk_size=512):
    """
    Loads a PDF file and splits its content into chunks of a specified size.

    Args:
        file (str): Path to the PDF file to be loaded.
        chunk_size (int): The maximum size of each chunk in characters (default is 512).

    Returns:
        List[Document]: A list of document chunks.
    """

    print(f'Loading and splitting into chunks: {pdf_file_name}')
    loader = PyPDFLoader(pdf_file_name)
    data = loader.load()

    # Split the content into chunks
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=64)
    chunks = text_splitter.split_documents(data)

    return chunks



In [ ]:
def expand_df(df):
    """
    Expands each row in the DataFrame by splitting PDF documents into chunks.

    Args:
        df (pd.DataFrame): DataFrame containing 'pdf_file_name', 'arxiv_id', 'title', 'summary',
                           'authors', and 'url' columns.

    Returns:
        pd.DataFrame: A new DataFrame where each row represents a chunk of the original document,
                      with additional metadata such as chunk identifiers and relationships to
                      adjacent chunks.
    """

    expanded_rows = []
    for idx, row in df.iterrows():
        try:
            chunks = load_and_chunk_pdf(row['pdf_file_name'])
        except Exception as e:
            print(f"Error processing file {row['pdf_file_name']}: {e}")
            continue

        for i, chunk in enumerate(chunks):
            prechunk_id = i-1 if i > 0 else ''
            postchunk_id = i+1 if i < len(chunks) - 1 else ''

            expanded_rows.append({
                'id': f"{row['arxiv_id']}#{i}",
                'title': row['title'],
                'summary': row['summary'],
                'authors': row['authors'],
                'arxiv_id': row['arxiv_id'],
                'url': row['url'],
                'chunk': chunk.page_content,
                'prechunk_id': '' if i == 0 else f"{row['arxiv_id']}#{prechunk_id}",  # Previous chunk ID
                'postchunk_id': '' if i == len(chunks) - 1 else f"{row['arxiv_id']}#{postchunk_id}"  # Next chunk ID
            })

    # Return a new expanded DataFrame which contents the meta data
    return pd.DataFrame(expanded_rows)


In [ ]:
expanded_df = expand_df(df)

In [ ]:
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv(), override=True)

In [ ]:
import os
from getpass import getpass

from semantic_router.encoders import OpenAIEncoder

os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY') or getpass('OpenAI API key: ')
encoder = OpenAIEncoder(name='text-embedding-3-small')

In [ ]:
encoder('hello hallo hola salut')

In [ ]:
dims = len(encoder('hello hallo hola salut')[0])
dims

In [ ]:
from pinecone import Pinecone, ServerlessSpec

api_key = os.getenv('PINECONE_API_KEY') or getpass('Pinecone API key: ')
pc = Pinecone(api_key=api_key)
spec = ServerlessSpec(
    cloud='aws',
    region='us-east-1'
)



In [ ]:
import time

index_name = 'langgraph-research-agent'

if index_name not in pc.list_indexes().names():
    pc.create_index(
        index_name,
        dimension=dims,  # Embedding dimension (1536)
        metric='cosine',
        spec=spec
    )

    while not pc.describe_index(index_name).status['ready']:
        time.sleep(1)

index = pc.Index(index_name)
time.sleep(1)
index.describe_index_stats()


In [ ]:
expanded_df.iloc[:5]

In [ ]:
from tqdm.auto import tqdm

data = expanded_df
batch_size = 64

for i in tqdm(range(0, len(data), batch_size)):
    i_end = min(len(data), i + batch_size)
    batch = data[i:i_end].to_dict(orient='records')

    metadata = [{
        'arxiv_id': r['arxiv_id'],
        'title': r['title'],
        'chunk': r['chunk'],
    } for r in batch]

    ids = [r['id'] for r in batch]
    chunks = [r['chunk'] for r in batch]
    embeds = encoder(chunks)
    index.upsert(vectors=zip(ids, embeds, metadata))


In [ ]:
index.describe_index_stats()

In [ ]:
import requests
arxiv_id = '1706.03762'
res = requests.get(f'https://arxiv.org/abs/{arxiv_id}')
res.text

In [ ]:
import re

abstract_pattern = re.compile(
    r'<blockquote class="abstract mathjax">\s*<span class="descriptor">Abstract:</span>\s*(.*?)\s*</blockquote>',
    re.DOTALL
)
re_match = abstract_pattern.search(res.text)
if re_match:
    print(re_match.group(1))
else:
    print('Abstract not found.')

In [ ]:
from langchain_core.tools import tool
import requests
import re

abstract_pattern = re.compile(
    r'<blockquote class="abstract mathjax">\s*<span class="descriptor">Abstract:</span>\s*(.*?)\s*</blockquote>',
    re.DOTALL
)

@tool('fetch_arxiv')
def fetch_arxiv(arxiv_id: str) -> str:
    '''Fetches the abstract from an ArXiv paper given its ArXiv ID.

    Args:
        arxiv_id (str): The ArXiv paper ID.

    Returns:
        str: The extracted abstract text from the ArXiv paper.
    '''

    res = requests.get(f'https://arxiv.org/abs/{arxiv_id}')

    re_match = abstract_pattern.search(res.text)

    return re_match.group(1) if re_match else 'Abstract not found.'


In [ ]:
arxiv_id = '1706.03762'
output = fetch_arxiv.invoke(input={'arxiv_id': arxiv_id})
print(output)

In [ ]:
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv(), override=True)

In [ ]:
from serpapi import GoogleSearch
import os
from getpass import getpass

serpapi_params = {
    'engine': 'google',
    'api_key': os.getenv('SERPAPI_KEY') or getpass('SerpAPI key: ')
}
search = GoogleSearch({
    **serpapi_params,
    'q': 'water',
    'num': 5
})

results = search.get_dict().get('organic_results', [])
formatted_results = '\n---\n'.join(
    ['\n'.join([x['title'], x['snippet'], x['link']]) for x in results]
)



In [ ]:
print(formatted_results)

In [ ]:
from serpapi import GoogleSearch

@tool('web_search')
def web_search(query: str) -> str:
    '''Finds general knowledge information using a Google search.

    Args:
        query (str): The search query string.

    Returns:
        str: A formatted string of the top search results, including title, snippet, and link.
    '''

    search = GoogleSearch({
        **serpapi_params,
        'q': query,
        'num': 5
    })

    results = search.get_dict().get('organic_results', [])
    formatted_results = '\n---\n'.join(
        ['\n'.join([x['title'], x['snippet'], x['link']]) for x in results]
    )
    return formatted_results if results else 'No results found.'


In [ ]:
output = web_search.invoke(input={'query': 'water on mars'})
print(output)

In [ ]:
def format_rag_contexts(matches: list) -> str:
    '''Formats the retrieved context matches into a readable string format.

    Args:
        matches (list): A list of matched documents with metadata.

    Returns:
        str: A formatted string of document titles, chunks, and ArXiv IDs.
    '''
    formatted_results = []
    for x in matches:
        text = (
            f"Title: {x['metadata']['title']}\n"
            f"Chunk: {x['metadata']['chunk']}\n"
            f"ArXiv ID: {x['metadata']['arxiv_id']}\n"
        )
        formatted_results.append(text)
    return '\n---\n'.join(formatted_results)


In [ ]:
from langchain_core.tools import tool

@tool
def rag_search_filter(query: str, arxiv_id: str) -> str:
    '''Finds information from the ArXiv database using a natural language query and a specific ArXiv ID.

    Args:
        query (str): The search query in natural language.
        arxiv_id (str): The ArXiv ID of the specific paper to filter by.

    Returns:
        str: A formatted string of relevant document contexts.
    '''

    xq = encoder([query])
    xc = index.query(vector=xq, top_k=6, include_metadata=True, filter={'arxiv_id': arxiv_id})
    return format_rag_contexts(xc['matches'])


In [ ]:
@tool('rag_search')
def rag_search(query: str) -> str:
    '''Finds specialist information on AI using a natural language query.

    Args:
        query (str): The search query in natural language.

    Returns:
        str: A formatted string of relevant document contexts.
    '''

    xq = encoder([query])
    xc = index.query(vector=xq, top_k=5, include_metadata=True)
    return format_rag_contexts(xc['matches'])


In [ ]:
from langchain_core.tools import tool

@tool
def final_answer(
    introduction: str,
    research_steps: str or list,
    main_body: str,
    conclusion: str,
    sources: str or list
) -> str:
    '''Returns a natural language response in the form of a research report.

    Args:
        introduction (str): A short paragraph introducing the user's question and the topic.
        research_steps (str or list): Bullet points or text explaining the steps taken for research.
        main_body (str): The bulk of the answer, 3-4 paragraphs long, providing high-quality information.
        conclusion (str): A short paragraph summarizing the findings.
        sources (str or list): A list or text providing the sources referenced during the research.

    Returns:
        str: A formatted research report string.
    '''

    if isinstance(research_steps, list):
        research_steps = '\n'.join([f'- {r}' for r in research_steps])

    if isinstance(sources, list):
        sources = '\n'.join([f'- {s}' for s in sources])

    return f'{introduction}\n\nResearch Steps:\n{research_steps}\n\nMain Body:\n{main_body}\n\n \
    Conclusion:\n{conclusion}\n\nSources:\n{sources}'


## 11 - Initializing the "Oracle" LLM

In [ ]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

system_prompt = (
    '''You are the oracle, the great AI decision-maker.
    Given the user's query, you must decide what to do with it based on the
    list of tools provided to you.

    If you see that a tool has been used (in the scratchpad) with a particular
    query, do NOT use that same tool with the same query again. Also, do NOT use
    any tool more than twice (i.e., if the tool appears in the scratchpad twice, do
    not use it again).

    You should aim to collect information from a diverse range of sources before
    providing the answer to the user. Once you have collected plenty of information
    to answer the user's question (stored in the scratchpad), use the final_answer tool.'''
)


prompt = ChatPromptTemplate.from_messages([
    ('system', system_prompt),
    MessagesPlaceholder(variable_name='chat_history'),
    ('user', '{input}'),
    ('assistant', 'scratchpad: {scratchpad}'),
])


In [ ]:
from langchain_core.messages import ToolCall, ToolMessage
from langchain_openai import ChatOpenAI
import os

llm = ChatOpenAI(
    model='gpt-4o',
    openai_api_key=os.environ['OPENAI_API_KEY'],
    temperature=0
)
tools = [
    rag_search_filter,
    rag_search,
    fetch_arxiv,
    web_search,
    final_answer
]

def create_scratchpad(intermediate_steps: list[ToolCall]) -> str:
    research_steps = []

    for i, action in enumerate(intermediate_steps):
        if action.log != 'TBD':
            research_steps.append(
                f'Tool: {action.tool}, input: {action.tool_input}\n'
                f'Output: {action.log}'
            )
    return '\n---\n'.join(research_steps)

oracle = (
    {
        'input': lambda x: x['input'],
        'chat_history': lambda x: x['chat_history'],
        'scratchpad': lambda x: create_scratchpad(intermediate_steps=x['intermediate_steps']),
    }
    | prompt
    | llm.bind_tools(tools, tool_choice='any')
)



## 12 -  Testing the Oracle and the Tools

In [ ]:
input = 'What is the ArXiv paper with the ID 2407.21783 all about?'
inputs = {
    'input': input,
    'chat_history': [],
    'intermediate_steps': [],
}
out = oracle.invoke(inputs)
print(out)



In [ ]:
out.tool_calls[0]['name']


In [ ]:
out.tool_calls[0]['args']

In [ ]:
def run_oracle(state: dict) -> dict:
    '''Runs the oracle and processes the output to extract tool information.

    Args:
        state (dict): The current state containing the 'intermediate_steps'.

    Returns:
        dict: A new state with updated 'intermediate_steps' including the tool action.
    '''

    print('run_oracle')
    print(f'intermediate_steps: {state["intermediate_steps"]}')

    out = oracle.invoke(state)
    tool_name = out.tool_calls[0]['name']
    tool_args = out.tool_calls[0]['args']
    action_out = AgentAction(
        tool=tool_name,
        tool_input=tool_args,
        log='TBD'
    )
    return {
        'intermediate_steps': [action_out]
    }


def router(state: dict) -> str:
    '''Determines the next tool to use based on the current state.

    Args:
        state (dict): The current state containing 'intermediate_steps'.

    Returns:
        str: The name of the tool to use next.
    '''

    if isinstance(state['intermediate_steps'], list):
        return state['intermediate_steps'][-1].tool
    else:
        print('Router invalid format')
        return 'final_answer'


tool_str_to_func = {
    'rag_search_filter': rag_search_filter,
    'rag_search': rag_search,
    'fetch_arxiv': fetch_arxiv,
    'web_search': web_search,
    'final_answer': final_answer
}

def run_tool(state: dict) -> dict:
    '''Executes the appropriate tool based on the current state.

    Args:
        state (dict): The current state containing the 'intermediate_steps'.

    Returns:
        dict: A new state with updated 'intermediate_steps' including the tool's result.
    '''

    tool_name = state['intermediate_steps'][-1].tool
    tool_args = state['intermediate_steps'][-1].tool_input

    print(f'{tool_name}.invoke(input={tool_args})')

    out = tool_str_to_func[tool_name].invoke(input=tool_args)

    action_out = AgentAction(
        tool=tool_name,
        tool_input=tool_args,
        log=str(out)
    )

    return {'intermediate_steps': [action_out]}

In [ ]:
from typing import TypedDict, Annotated, List
from langchain_core.agents import AgentAction
from langchain_core.messages import BaseMessage
import operator

class AgentState(TypedDict):
    '''Represents the state of an agent.'''

    input: str
    chat_history: List[BaseMessage]
    intermediate_steps: Annotated[List[tuple[AgentAction, str]], operator.add]

In [ ]:
from langgraph.graph import StateGraph, END

graph = StateGraph(AgentState)
graph.add_node('oracle', run_oracle)
graph.add_node('rag_search_filter', run_tool)
graph.add_node('rag_search', run_tool)
graph.add_node('fetch_arxiv', run_tool)
graph.add_node('web_search', run_tool)
graph.add_node('final_answer', run_tool)

graph.set_entry_point('oracle')
graph.add_conditional_edges(source='oracle', path=router)
for tool_obj in tools:
    if tool_obj.name != 'final_answer':
        graph.add_edge(tool_obj.name, 'oracle')

graph.add_edge('final_answer', END)
runnable = graph.compile()


In [ ]:
from IPython.display import Image, display
display(Image(runnable.get_graph().draw_mermaid_png()))

In [ ]:
output = runnable.invoke({
    'input': 'Tell me something interesting about Dynamic Backtracking AI and LLMs',
    'chat_history': [],
})


In [ ]:
def build_report(output: dict) -> str:
    '''Builds a formatted report based on the oracle's output.

    Args:
        output (dict): A dictionary containing the various sections of the report (graph's output).

    Returns:
        str: A formatted string containing the full research report.
    '''
    research_steps = output['research_steps']
    if isinstance(research_steps, list):
        research_steps = '\n'.join([f'- {r}' for r in research_steps])

    sources = output['sources']
    if isinstance(sources, list):
        sources = '\n'.join([f'- {s}' for s in sources])

    return f"""
        INTRODUCTION
        ------------
        {output['introduction']}

        RESEARCH STEPS
        --------------
        {research_steps}

        REPORT
        ------
        {output['main_body']}

        CONCLUSION
        ----------
        {output['conclusion']}

        SOURCES
        -------
        {sources}
    """


In [ ]:
# Run the graph with input.
output = runnable.invoke({
    'input': 'Tell me something interesting about Dynamic Backtracking AI and LLMs',
    'chat_history': [],
})

In [ ]:
output['intermediate_steps'][-1].tool_input

In [ ]:
output = runnable.invoke({
    'input': 'tell me about FIFA World Cup 26',
    'chat_history': []
})


In [ ]:
report = build_report(
    output=output['intermediate_steps'][-1].tool_input
)

print(report)

In [ ]:
output = runnable.invoke({
    'input': 'Create a summary about this AxXiv paper with the ID 2409.17990',
    'chat_history': []
})


In [ ]:
report = build_report(
    output=output['intermediate_steps'][-1].tool_input
)

print(report)

In [ ]:
output = runnable.invoke({
    'input': 'Create a summary about the future of LLM Agents.',
    'chat_history': []
})

report = build_report(
    output=output['intermediate_steps'][-1].tool_input
)

print(report)

run_oracle
intermediate_steps: []
rag_search.invoke(input={'query': 'future of LLM Agents'})
run_oracle
intermediate_steps: [AgentAction(tool='rag_search', tool_input={'query': 'future of LLM Agents'}, log='TBD'), AgentAction(tool='rag_search', tool_input={'query': 'future of LLM Agents'}, log='Title: On Planning while Learning\nChunk: one go al/, if an e/\x0ecient satisfactory multi/-agent plan for achievingthese go als exists/, then ther e exists such an e/\x0ecient satisfactory multi/-agent plan that c anb e enc o de d in p olynomial sp ac e/, and b e veri/\x0ce d in p olynomial time/.Pro of/: In this case eac h agen t kno ws the goal of the other agen t/, and hence it is clear thatit migh t learn only facts ab out the p ossible initial states and b eha viors/.Giv en that there is only a p olynomial n um b er of p ossible initial states and en\nArXiv ID: 9409101v1\n\n---\nTitle: A Market-Oriented Programming Environment and its Application to\n  Distributed Multicommodity Flow Probl